# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/city_weather_data.csv")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,fale old settlement,-9.3852,-171.2468,28.63,75,88,4.54,TK,1689908719
1,blackmans bay,-43.0167,147.3167,11.40,59,3,2.45,AU,1689908719
2,yellowknife,62.4560,-114.3525,26.29,44,100,4.12,CA,1689908719
3,bilibino,68.0546,166.4372,23.36,45,3,2.19,RU,1689908720
4,hassi messaoud,31.6804,6.0729,32.92,10,0,1.54,DZ,1689908720


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(x='Lng', y='Lat', size='Humidity', title='City Humidity Map', hover_cols=['City', 'Country'],color="City",tiles="OSM",geo=True)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

In [9]:
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] > 20) & (city_data_df['Max Temp'] < 30) & (city_data_df['Wind Speed'] < 10) & (city_data_df['Cloudiness'] < 50)].dropna()

# Drop any rows with null values
ideal_weather_df.dropna(inplace=True)

# Display sample data
ideal_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,bilibino,68.0546,166.4372,23.36,45,3,2.19,RU,1689908720
6,lockwood,45.8191,-108.4149,26.94,41,40,5.14,US,1689908720
16,albany,42.6001,-73.9662,21.84,79,40,1.79,US,1689908599
18,bayanhongor,46.7167,100.1167,20.02,34,36,2.00,MN,1689908722
21,georgetown,5.4112,100.3354,29.97,79,20,3.13,MY,1689908576


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
3,bilibino,RU,68.0546,166.4372,45,
6,lockwood,US,45.8191,-108.4149,41,
16,albany,US,42.6001,-73.9662,79,
18,bayanhongor,MN,46.7167,100.1167,34,
21,georgetown,MY,5.4112,100.3354,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000  # assuming you want to search within a 10,000 meters radius
params = {
    'apiKey': geoapify_key,
    'categories': "accommodation.hotel",
    'limit': 20,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

# Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

# Configure the map plot
map_plot_with_hotels = hotel_df.hvplot.points(x='Longitude', y='Latitude', size='Humidity', title='City Humidity Map with Hotels', hover_cols=['City', 'Country', 'Hotel Name'], geo=True)

# Display the map with hotels
map_plot_with_hotels

Starting hotel search
bilibino - nearest hotel: No hotel found
lockwood - nearest hotel: Holiday Inn Express
albany - nearest hotel: No hotel found
bayanhongor - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
nova sintra - nearest hotel: Residência Ka Dencho
port mathurin - nearest hotel: Escale Vacances
severo-yeniseyskiy - nearest hotel: Актолик
klyuchi - nearest hotel: No hotel found
kailua-kona - nearest hotel: Kona Seaside Hotel
sao miguel do tapuio - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
la'ie - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
pacasmayo - nearest hotel: Playa Linda
tazacorte - nearest hotel: App Leyma
demopolis - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
utrik - nearest hotel: No hotel found
tamuin - nearest hotel: Hotel Villa Guerrero
port isabel - nearest hotel: Queen Isabel Inn
jaffa - nearest hotel: מלון מרגוזה
kapshagay - nearest hotel: Казино

DataError: Supplied data does not contain specified dimensions, the following dimensions were not found: ['Longitude', 'Latitude']

PandasInterface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot_with_hotels = hotel_df.hvplot.points(x='Longitude', y='Latitude', size='Humidity', title='City Humidity Map with Hotels', hover_cols=['City', 'Country', 'Hotel Name'], color="City",tiles="OSM",geo=True)

# Display the map with hotels
map_plot_with_hotels